In [1]:
import re
import os

import pandas as pd
from pymystem3 import Mystem
from nltk.corpus import stopwords

In [2]:
mystem = Mystem()
swords = set(stopwords.words('russian'))

In [16]:
# Преобразование файлов pkl к csv с стандартизацией текстовых данных

def lemm_file(file_path):
    try:
        df = pd.DataFrame(pd.read_pickle(f'ria_economy/raw_cites/{file_path}'))
        # df = pd.DataFrame(pd.read_pickle(f'ria_economy/splitted file/{file_path}'))
        df['tp'] = df['data_or_ex'].map(lambda x: isinstance(x, tuple))
        df.drop(df[df.tp != True].index, inplace=True)
        df[['date', 'title', 'announce', 'text']] = pd.DataFrame(df['data_or_ex'].tolist(), index=df.index)
        df.drop(columns=['data_or_ex', 'tp'], inplace=True)
        # df.drop(columns=['data_or_ex'], inplace=True)
        cols = ['title', 'announce', 'text']
        for col in cols:
            df[col] = df[col].str.lower()
            df[col] = df[col].str.replace('\n', '')
            df[col] = df[col].str.replace('\W+', ' ', regex=True)
            res = []
            doc = []
            large_str = ' '.join([txt + ' splitter ' for txt in df[col]])
            large_str = mystem.lemmatize(large_str)

            for word in large_str:
                if word.strip() != '' and word not in swords:
                    if word == 'splitter':
                        res.append(doc)
                        doc = []
                    else:
                        doc.append(word)
            del large_str
            del doc
            res = [' '.join(lst) for lst in res]
            df[col] = res
            print(f'{col} ready')
        return df
    except KeyboardInterrupt:
        raise
    except Exception as ex:
        print(ex)
        return None

In [18]:
# for_split = pd.DataFrame(pd.read_pickle('ria_economy/raw_cites/13726.pkl'))
# for_split['tp'] = for_split['data_or_ex'].map(lambda x: isinstance(x, tuple))
# for_split.drop(for_split[for_split.tp != True].index, inplace=True)
# for_split.drop(columns=['tp'], inplace=True)
# for_split.reset_index(inplace=True, drop=True)

# c = 0

# while c < 12953:
#     a = 1000
#     if c + a > 12953:
#         a = 953
#     tmp = for_split.iloc[c: c+a, :]
#     tmp.to_pickle(f'{c+a}.pkl')
#     c += a

In [17]:
# Запускаем функцию

files = sorted(os.listdir('ria_economy/raw_cites'))
files = [file for file in files if 'skipped' in file]
c = 1

for file in files:
    # Изнчачально у меня был большой файл, который не открывался и это временная заглушка была, потом я его побил кодом ячейкой выше
    if file == '13726.pkl':
        continue
    print(f'file {file} in work')
    df = lemm_file(f'/{file}')
    if df is None:
        continue
    df.to_csv(f'ria_economy/prep_cites/{file[:-4]}.csv', index=False)
    print(f'{c} из {len(files)} готов')
    c += 1

file 0_skipped.pkl in work
title ready
announce ready
text ready
1 из 25 готов
file 1000_skipped.pkl in work
title ready
announce ready
text ready
2 из 25 готов
file 1200_skipped.pkl in work
title ready
announce ready
text ready
3 из 25 готов
file 1400_skipped.pkl in work
title ready
announce ready
text ready
4 из 25 готов
file 1600_skipped.pkl in work
title ready
announce ready
text ready
5 из 25 готов
file 1800_skipped.pkl in work
title ready
announce ready
text ready
6 из 25 готов
file 2000_skipped.pkl in work
title ready
announce ready
text ready
7 из 25 готов
file 200_skipped.pkl in work
title ready
announce ready
text ready
8 из 25 готов
file 2200_skipped.pkl in work
title ready
announce ready
text ready
9 из 25 готов
file 2400_skipped.pkl in work
title ready
announce ready
text ready
10 из 25 готов
file 2600_skipped.pkl in work
title ready
announce ready
text ready
11 из 25 готов
file 2800_skipped.pkl in work
title ready
announce ready
text ready
12 из 25 готов
file 3000_skipped

In [3]:
# Преобразование файлов Телеграмма к csv в стандартизованном виде

def preprocess_tg(file):
    try:
        df = pd.read_json(f'Telegram data/{file}')[['date', 'message', 'views', 'forwards', 'fwd_from']].dropna(subset='message')
        n_rows = df.shape[0]
        c = 0
        cols = ['message']
        for col in cols:
            df[col] = df[col].str.replace('\n', '')
            df[col] = df[col].str.replace('\W+', ' ', regex=True)
            df[col] = df[col].str.lower()
            while c < n_rows:
                res = []
                doc = []
                large_str = ' '.join([txt + ' splitter ' for txt in df[col][c:c+1000]])
                large_str = mystem.lemmatize(large_str)

                for word in large_str:
                    if word.strip() != '' and word not in swords:
                        if word == 'splitter':
                            if len(doc) == 0:
                                doc.append('')
                            res.append(doc)
                            doc = []
                        else:
                            doc.append(word)
                del large_str
                del doc
                res = [' '.join(lst) for lst in res]
                df[col][c:c+1000] = res
                c += 1000
            print(f'{col} ready')
        return df
    except KeyboardInterrupt:
        raise
    except Exception as ex:
        print(ex)
        return None

In [4]:
files = sorted(os.listdir('Telegram data'))[1:]
c = 1

for file in files:
    print(f'file {file} in work')
    df = preprocess_tg(f'/{file}')
    df.to_csv(f'Telegram prep/{file[:-4]}csv', index=False)
    print(f'{c} из {len(files)} готов')
    c += 1

file markettwits_messages.json in work


/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_1163/1199387682.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  large_str = ' '.join([txt + ' splitter ' for txt in df[col][c:c+1000]])
/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_1163/1199387682.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][c:c+1000] = res
/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_1163/1199387682.py:29: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be

message ready
12 из 3 готов
file nedvizhna24_messages.json in work


/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_1163/1199387682.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][c:c+1000] = res


message ready
13 из 3 готов
file smartlab_messages.json in work


/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_1163/1199387682.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  large_str = ' '.join([txt + ' splitter ' for txt in df[col][c:c+1000]])
/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_1163/1199387682.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][c:c+1000] = res
/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_1163/1199387682.py:29: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be

message ready
14 из 3 готов


In [2]:
pd.read_csv('Telegram prep/banksta_messages.csv')

,date,message,views,forwards,fwd_from
0,2022-11-30 18:32:01+00:00,евросоюз угрожать забанить твиттер европа илон...,27237.0,208.0,NaN
1,2022-11-30 17:40:01+00:00,помогать бизнес расти помощь медийный рекламым...,35658.0,10.0,NaN
2,2022-11-30 17:38:44+00:00,общий стоимость лондонский недвижимость которы...,33211.0,43.0,NaN
3,2022-11-30 17:31:04+00:00,криптовалютный биржа kraken сокращать персонал...,34631.0,72.0,NaN
4,2022-11-30 17:00:02+00:00,почта банк выплачивать клиент 25 млрд рубль мо...,38149.0,11.0,"{'_': 'MessageFwdHeader', 'date': '2022-11-30T..."
...,...,...,...,...,...
29442,2017-09-04 10:23:29+00:00,фсб вести разработка экс депутат андрей скотат...,33.0,1.0,"{'_': 'MessageFwdHeader', 'date': '2017-09-04T..."
29443,2017-09-04 08:42:24+00:00,банковский работник высказываться открытие емк...,66.0,0.0,"{'_': 'MessageFwdHeader', 'date': '2017-09-03T..."
29444,2017-09-04 07:31:14+00:00,команда бывший владелец группа конверс владими...,414.0,1.0,NaN
29445,2017-09-03 17:35:27+00:00,тело близкий семья билалов глава компания куро...,85.0,0.0,"{'_': 'MessageFwdHeader', 'date': '2017-09-03T..."
